In [1]:
import glob
import pandas as pd
import numpy as np
from IPython.core import display as ICD

In [2]:
import re
def standardize_track(track):
    if isinstance(track, str):
        if (re.search('apps', track, re.IGNORECASE)):
            return 'Apps'
        elif (re.search('intro', track, re.IGNORECASE)):
            return 'Intro'
        elif (re.search('games', track, re.IGNORECASE)):
            return 'Games'
        elif (re.search('vr', track, re.IGNORECASE)):
            return 'VR'
    return track

# 2016

## Week 8

In [3]:
df_2016_week8 = pd.read_csv('../Datasets/SA Feedback Surveys_FINAL/2016/Week 8 Feedback (2016, incomplete) - results.csv')
df_2016_week8 = df_2016_week8.rename({
    'How likely is it that you would recommend the Make School Summer Academy to a friend?': 'Overall Rating',
    'location': 'Location',
    'track': 'Track'
}, axis=1)

df_2016_week8 = df_2016_week8[['Location', 'Track', 'Overall Rating']]

df_2016_week8['Track'] = df_2016_week8['Track'].map(standardize_track)
df_2016_week8['Week'] = 8
df_2016_week8['Year'] = 2016

df_2016_week8.head()

,Location,Track,Overall Rating,Week,Year
0,New York,Apps,10,8,2016
1,San Francisco,Intro,10,8,2016
2,New York,Intro,10,8,2016
3,Sunnyvale,Apps,9,8,2016
4,San Francisco,Intro,3,8,2016


## Anon

In [4]:
files = glob.glob('../Datasets/SA Feedback Surveys_FINAL/2016/Anon *.csv')
datasets = { file: pd.read_csv(file) for file in files }

In [5]:
df_2016_anons = []

for dataset in datasets:
    name = dataset.split('/')[-1]
    
    name_split = name.split()
    df = datasets[dataset]
        
    # Fuzzy match rating columns
    overall_ratings = df.filter(like='overall').iloc[:,0]
    pacing_ratings = df.filter(like='pac').iloc[:,0]
    
    # Convert to numbers, drop nan, convert to int and double
    overall_ratings = pd.to_numeric(overall_ratings, errors='coerce').dropna().astype(int).apply(lambda x: x * 2)
    pacing_ratings = pd.to_numeric(pacing_ratings, errors='coerce').dropna().astype(int).apply(lambda x: x * 2)
    
    df_week = pd.DataFrame({
        'Overall Rating': overall_ratings,
        'Pacing Rating': pacing_ratings,
        'Week': int(name_split[2]),
        'Location': name_split[-1].split('.')[0],
        'Year': 2016
    })
    df_2016_anons.append(df_week)

# 2017

In [6]:
df_2017 = pd.read_csv('../Datasets/SA Feedback Surveys_FINAL/2017/Student Feedback Surveys-Superview.csv')

df_2017['Week'] = df_2017['Week'].apply(lambda x: int(x.split()[-1]))
df_2017['Track'] = df_2017['Track'].map(standardize_track)
df_2017['Overall Rating'] = pd.to_numeric(df_2017['Rating (Num)'], errors='coerce')
df_2017['Pacing Rating'] = df_2017['Schedule Pacing'].map({
    'Way too slow': 2,
    'A little too slow': 4,
    'Just right': 6,
    'A little too fast': 8,
    'Way too fast': 10
})
df_2017['Year'] = 2017
ICD.display(df_2017['Schedule Pacing'].unique())

df_2017 = df_2017.drop(['ID', 'Rating (Num)', 'Schedule Pacing'], axis=1)

df_2017.head()

array(['Just right', 'A little too fast', 'Way too slow',
       'A little too slow', 'Way too fast', nan], dtype=object)

,Location,Track,Week,Overall Rating,Pacing Rating,Year
0,San Francisco,Apps,1,3.0,6.0,2017
1,Los Angeles,Apps,1,4.0,8.0,2017
2,San Francisco,Games,1,4.0,2.0,2017
3,NaN,NaN,2,4.0,8.0,2017
4,New York City,Apps,1,4.0,6.0,2017


# Combined

In [7]:
df = pd.concat([df_2016_week8, *df_2016_anons, df_2017], sort=False)
df.head()

,Location,Track,Overall Rating,Week,Year,Pacing Rating
0,New York,Apps,10.0,8,2016,NaN
1,San Francisco,Intro,10.0,8,2016,NaN
2,New York,Intro,10.0,8,2016,NaN
3,Sunnyvale,Apps,9.0,8,2016,NaN
4,San Francisco,Intro,3.0,8,2016,NaN
